In [1]:
def func():
    global var
    var = 'hi'

In [2]:
var

NameError: name 'var' is not defined

In [3]:
func()

In [4]:
var

'hi'

In [5]:
a = 10

In [6]:
def func3():
    print(a)
    a = '123'

In [7]:
func3()

UnboundLocalError: cannot access local variable 'a' where it is not associated with a value

In [8]:
y = 20
def maria():
    x = 10

    def rose():

        def sina():
            nonlocal x
            global y
            x = 'abc'
            y = 'def'
        
        sina()
    rose()
    print(x)
maria()
print(y)

abc
def


## Closures

In [9]:
def outer():
    x = "python"
    def inner():
        return x
    return inner

In [10]:
fn = outer()

In [11]:
fn.__code__.co_freevars

('x',)

In [12]:
fn.__closure__

(<cell at 0x000001D488B16740: str object at 0x000001D48578C2B0>,)

In [13]:
def outer():
    x = [1, 2, 3]
    print(hex(id(x)))
    def inner():
        print(hex(id(x)))
    return inner
fn = outer()


0x1d488fdeb80


In [14]:
fn()

0x1d488fdeb80


In [15]:
fn.__code__.co_freevars

('x',)

In [16]:
def create_adders():
    adders = []
    for i in range(4):
        adders.append(lambda x, y=i: x + i)
    return adders

In [13]:
def averager():
    total = 0
    count = 0
    def add(number):
        nonlocal total
        nonlocal count
        total += number
        count += 1
        return total / count
    return add

In [14]:
a = averager()

In [16]:
a(20)

30.0

In [17]:
from time import perf_counter

In [18]:
perf_counter()

1475452.5641454

In [31]:
def timer():
    start = perf_counter()
    def poll():
        return perf_counter() - start
    return poll

In [32]:
p = timer()

In [51]:
p()

2064.352222000016

In [ ]:
def counter(initial_value=0):
    count = initial_value
    def inc(n=1):
        nonlocal count
        count += n
        return count
    return inc

In [41]:
c = counter()

In [50]:
c(12)

24

In [18]:
def func_counter(fn):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f"{fn.__name__} has been called {cnt} times")
        return fn(*args, **kwargs)
    return inner

In [55]:
def add(a, b):
    return a + b

In [57]:
f_cnt = func_counter(add)

In [67]:
f_cnt(12, 25)

add has been called 10 times


37

In [82]:
func_counts = {}

In [83]:
def func_counter(fn, func_counts):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        func_counts[fn.__name__] = cnt
        return fn(*args, **kwargs)
    return inner

In [85]:
counted_add = func_counter(add, func_counts)

In [80]:
counted_add(1, 2)

3

In [81]:
func_counts

{'add': 2}

In [113]:
def factorialer():
    cache = {}
    def inner(n):
        return 1 if n < 2 else cache.get(n) or (n * inner(n - 1))
    return inner

In [110]:
def factorialer():
    cache = {}
    def inner(n):
        if n < 2:
            return 1
        if n in cache:
            return cache[n]
        return n * inner(n - 1)
    return inner

In [114]:
fact = factorialer()

In [116]:
fact(21)

51090942171709440000

## Decorators

In [19]:
@func_counter
def mult(a, b):
    return a * b

In [25]:
mult(2, 6)

mult has been called 6 times


12

In [26]:
mult.__name__

'inner'

In [27]:
from functools import wraps

In [28]:
def func_counter(fn):
    cnt = 0
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f"{fn.__name__} has been called {cnt} times")
        return fn(*args, **kwargs)
    return inner

In [29]:
@func_counter
def mult(a, b):
    return a * b

In [30]:
mult.__name__

'mult'

#### Decorator exercises

Write a decorator @enforce_types(int, int) that ensures the first two arguments passed to a function are of type int, or raises a TypeError.

In [34]:
def enforce_types(fn, a, b):
    @wraps(fn)
    def inner(*args, **kwargs):
        if not (args and len(args) >= 2 and type(args[0]) == a and type(args[1]) == b):
            raise TypeError(f'First 2 arguments need to be of type {a} and {b}')
        return fn(*args, **kwargs)
    return inner

In [35]:
@enforce_types(int, int)
def add(a, b):
    return a + b

TypeError: enforce_types() missing 1 required positional argument: 'b'

A decorator that allows a function to run only once. Subsequent calls return the result of the first call.

In [ ]:
def run_once(fn):
    has_ran = False
    result = None

    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal result, has_ran
        if not has_ran:
            result = fn(*args, **kwargs)
            has_ran = True
        return result
    return inner


In [39]:
@run_once
def sq(a):
    return a ** 2

In [43]:
sq(6)

4

Write a decorator that caches the result of a function for a given input (memoization), like @cache.

In [85]:
def cache(fn):
    cache_dict = {}

    @wraps(fn)
    def inner(*args, **kwargs):
        if (x := (frozenset(args), frozenset(kwargs.keys()), frozenset(kwargs.values()))) in cache_dict:
            return cache_dict[x]
        result = fn(*args, **kwargs)
        cache_dict[(frozenset(args), frozenset(kwargs.keys()), frozenset(kwargs.values()))] = result
        print(cache_dict)
        return result
    
    return inner


In [86]:
@cache
def divstringer(a, b, s=''):
    return s + str(a // b)

In [88]:
divstringer(111, 22, s='asbc')

{(frozenset({2, 111}), frozenset({'s'}), frozenset({'abc'})): 'abc55', (frozenset({22, 111}), frozenset({'s'}), frozenset({'asbc'})): 'asbc5'}


'asbc5'

In [91]:
def timed(fn):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start

        args_ = [str(a) for a in args]
        kwargs_ = [f'{k}={v}' for k, v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ', '.join(all_args)

        print(f'{fn.__name__}({args_str}) took {elapsed:.6f}s to run')
        
        return result
    
    return inner

In [97]:
def fib(n):
    if n <= 2: 
        return 1
    return fib(n - 1) + fib(n - 2)

In [99]:
@timed
def recursive_fib(n):
    return fib(n)

In [107]:
recursive_fib(40)

recursive_fib(40) took 10.928761s to run


102334155

In [122]:
@timed
def loop_fib(n):
    fib_1 = 1
    fib_2 = 1
    for i in range(n-2):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
    return fib_2


In [123]:
loop_fib(40)

loop_fib(40) took 0.000004s to run


102334155

In [8]:
from functools import reduce

In [138]:
@timed
def reduce_fib(n):
    return reduce(lambda x, y: (x[1], x[0] + x[1]), range(n - 1), (0, 1))[1]

In [141]:
reduce_fib(40)

reduce_fib(40) took 0.000008s to run


102334155

In [ ]:
def avg_timed(fn, n):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        total = 0
        count = 0
        for i in range(n):
            start = perf_counter()
            result = fn(*args, **kwargs)
            end = perf_counter()
            elapsed = end - start
            total += elapsed
            count += 1

        args_ = [str(a) for a in args]
        kwargs_ = [f'{k}={v}' for k, v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ', '.join(all_args)

        print(f'{fn.__name__}({args_str}) took {total / count :.6f}s to run')

        return result
    
    return inner

In [ ]:
@avg_timed(15)
def recursive_fib(n):
    return fib(n)

In [155]:
recursive_fib(32)

recursive_fib(32) took 0.239005s to run


2178309

In [2]:
def logged(fn):
    from functools import wraps
    from datetime import datetime

    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now()
        result = fn(*args, **kwargs)
        print(f'{run_dt}: called {fn.__name__}')
        return result
    
    return inner

In [5]:
@logged
def func1():
    pass

In [6]:
func1()

2025-06-24 13:00:26.365636: called func1


In [7]:
def timed(fn):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start

        args_ = [str(a) for a in args]
        kwargs_ = [f'{k}={v}' for k, v in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ', '.join(all_args)

        print(f'{fn.__name__}({args_str}) took {elapsed:.6f}s to run')
        
        return result
    
    return inner

In [26]:
@logged
@timed
def fact(n):
    return reduce(lambda x, y: x * y, range(1, n+1))

In [27]:
fact(1500)

fact(1500) took 0.000588s to run
2025-06-24 13:05:30.800586: called fact


4811997796779774860166990093581379781834808040672613808130855941163057518900109559129223058520673385186846400961934358519405209112461816627027148188139333143162796281029984414933378904468939551048716787976932530369947046782923439926332654565286074860507574636692832360664549227754112008343808672736937788767600021140531848024435420741960486417696995058143522219885119456898409570594554958905456832179233891914944298591995773479295940249909684564302040186938117560396442433322211412597437481780424263330976980429395287003461935412501421004564766406324016200756010866529056864612834255714735098535872415462325337186747076512042207386796393577525869210975304176209434356905049747035353176448150317475091185823090699836106608478775831611058573601336537743186073857226132573823365683527194735269518086557304383402795553901276548937264504250440659775235748193153287235663541122457833404052229474640282958545847870877834637943186236882481900917709144403488594139431934391022316865586976179966907505952760850

In [ ]:
def fib(n):
    print(f'calculating {n}')
    return 1 if n <= 2 else fib(n - 1) + fib(n - 2)

In [44]:
class Fib:
    def __init__(self):
        self.cache = {1: 1, 2: 1}

    def __call__(self, n):
        if n in self.cache:
            return self.cache[n]
        self.cache[n] = (res := self(n - 1) + self(n - 2))
        return res

In [45]:
f = Fib()

In [46]:
f(116)

781774079430987230203437

In [51]:
def fibber(cache={1: 1, 2: 1}):
    
    def inner(n):
        if n not in cache:
            cache[n] = inner(n - 1) + inner(n - 2)
        return cache[n]
    
    return inner


In [52]:
f2 = fibber()

In [53]:
f2(116)

781774079430987230203437

In [54]:
def memoize_single_arg(fn):
    cache = dict()

    def inner(x):
        if x not in cache:
            cache[x] = fn(x)
        return cache[x]

    return inner

In [ ]:
def memoize_no_kwargs(fn):
    cache = dict()

    def inner(*args):
        if x:=tuple(args) not in cache:
            cache[x] = fn(x)
        return cache[x]

    return inner

In [ ]:
@memoize_single_arg
def fib(n):
    print(f'calculating {n}')
    return 1 if n <= 2 else fib(n - 1) + fib(n - 2)

In [61]:
fib(166)

22002056689466296922983322104048463

In [ ]:
from functools import lru_cache

In [72]:
@lru_cache(maxsize=8)
def fib(n):
    print(f'calculating {n}')
    return 1 if n <= 2 else fib(n - 1) + fib(n - 2)

In [79]:
fib(10)

calculating 10
calculating 9
calculating 8
calculating 7
calculating 6


55

In [12]:
def avg_timed(n):
    def dec(fn):
        from time import perf_counter
        from functools import wraps

        @wraps(fn)
        def inner(*args, **kwargs):
            total = 0
            for i in range(n):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                elapsed = end - start
                total += elapsed

            args_ = [str(a) for a in args]
            kwargs_ = [f'{k}={v}' for k, v in kwargs.items()]
            all_args = args_ + kwargs_
            args_str = ', '.join(all_args)

            print(f'{fn.__name__}({args_str}) took {total / n :.6f}s to run')

            return result
        
        return inner
    return dec

In [13]:
def fib(n):
    return 1 if n <= 2 else fib(n - 1) + fib(n - 2)

In [9]:
@avg_timed(15)
def recursive_fib(n):
    return fib(n)

In [11]:
recursive_fib(32)

recursive_fib(32) took 0.320029s to run


2178309

In [23]:
class AvgTimed:
    def __init__(self, n):
        self.n = n

    def __call__(self, fn):
        from time import perf_counter
        from functools import wraps
        
        @wraps(fn)
        def inner(*args, **kwargs):
            total = 0
            for i in range(self.n):
                start = perf_counter()
                result = fn(*args, **kwargs)
                end = perf_counter()
                elapsed = end - start
                total += elapsed

            args_ = [str(a) for a in args]
            kwargs_ = [f'{k}={v}' for k, v in kwargs.items()]
            all_args = args_ + kwargs_
            args_str = ', '.join(all_args)

            print(f'{fn.__name__}({args_str}) took {total / self.n :.6f}s to run')

            return result
        
        return inner
        
        

In [25]:
@AvgTimed(15)
def recursive_fib(n):
    return fib(n)

In [26]:
recursive_fib(30)

recursive_fib(30) took 0.125212s to run


832040

In [27]:
from fractions import Fraction

In [32]:
f = Fraction(2, 3)

In [39]:
def dec_speak(cls):
    cls.speak = lambda self, message: f"message: {message}"

In [41]:
dec_speak(Fraction)

In [42]:
f.speak('ori mesamedisgan salami')

'message: ori mesamedisgan salami'

In [13]:
from datetime import datetime, timezone

In [6]:
def info(obj):
        result = []
        result.append(f"time: {datetime.now(timezone.utc)}")
        result.append(f"class: {obj.__class__.__name__}")
        result.append(f"id: {hex(id(obj))}")
        for k, v in vars(obj).items():
            result.append(f"{k}: {v}")
        return result

In [7]:
def dec_debug(cls):
    cls.debug = info
    return cls

In [54]:
@dec_debug
class Person:
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year

    def say_hi():
        print('hi')

In [55]:
p = Person('nika', 2001)

In [60]:
p.debug()

['time: 2025-06-29 08:16:33.455765+00:00',
 'class: Person',
 'id: 0x24a21363d10',
 'name: nika',
 'birth_year: 2001']

In [8]:
@dec_debug
class Automobile:
    def __init__(self, make, model, year, top_speed):
        self.make = make
        self.model = model
        self. year = year
        self.top_speed = top_speed
        self._speed = 0

    @property
    def speed(self):
        return self._speed
    
    @speed.setter
    def speed(self, value):
        if value <= self.top_speed:
            self._speed = value
        else:
            raise ValueError('Speed can not exceed top speed.')

In [9]:
my_car = Automobile('Nissan', 'Rogue Sport', 2017, 220)

In [16]:
my_car.speed = 200

In [17]:
my_car.debug()

['time: 2025-06-29 09:03:32.338565+00:00',
 'class: Automobile',
 'id: 0x19e525cb450',
 'make: Nissan',
 'model: Rogue Sport',
 'year: 2017',
 'top_speed: 220',
 '_speed: 200']

In [18]:
from math import sqrt

In [ ]:
def with_lt_eq(obj):
    if '__lt__' in dir(obj) and '__eq__' in dir(obj):
        obj.__ne__ = lambda self, other: not self == other
        obj.__le__ = lambda self, other: self < other or self == other
        obj.__ge__ = lambda self, other: not self < other
        obj.__gt__ = lambda self, other: not self <= other
    return obj

In [33]:
@with_lt_eq
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)
    
    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            False

    def __lt__(self, other):
        if isinstance(other, Point):
            return abs(self) < abs(other)
        else:
            return NotImplementedError

In [34]:
p1, p2 = Point(1, 1), Point(2, 2)

In [ ]:
p1 != p2

True

: 

In [ ]:
from functools import total_ordering

### Single dispatch generic functions

In [1]:
from html import escape
from decimal import Decimal

In [3]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return f"{a}(<i>{str(hex(a))}</i>)"

def html_real(r):
    return f"{a:.2f}"

def html_str(s):
    return s.replace('\n', '<br/>\n')

def html_list(l):
    items = (f"<li>{html_escape(item)}</li>" for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = (f"<li>{k}={v}</li>" for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [ ]:
def htmlize(arg): 
    registry = {
        object: html_escape,
        int: html_int,
        float: html_int,
        Decimal: html_int,
        str: html_str,
        list: html_list,
        tuple: html_list,
        dict: html_dict,
    }

    fn = registry.get(type(arg), registry[object])
    
    return fn

In [3]:
def single_dispatch(fn):
    registry = {}

    registry[object] = fn
    def inner(arg):
        return registry[object](arg)
    
    return inner

In [4]:
@single_dispatch
def html_escape(arg):
    return escape(str(arg))

In [12]:
def single_dispatch(fn):
    registry = {}
    registry[object] = fn

    def dec(arg):
        return registry.get(type(arg), registry[object])(arg)

    def register(type_):

        def inner(fn):
            registry[type_] = fn
            return fn
        return inner
    
    dec.register = register
    dec.registry = registry

    return dec

In [13]:
@single_dispatch
def htmlize(a):
    return escape(str(a))


In [15]:
htmlize.registry

{object: <function __main__.htmlize(a)>}

In [16]:

@htmlize.register(int)
def html_int(a):
    return f"{a}(<i>{str(hex(a))}</i>)"

@htmlize.register(float)
@htmlize.register(Decimal)
def html_real(r):
    return f"{a:.2f}"

@htmlize.register(str)
def html_str(s):
    return s.replace('\n', '<br/>\n')

@htmlize.register(list)
@htmlize.register(set)
@htmlize.register(tuple)
def html_sequence(l):
    items = (f"<li>{html_escape(item)}</li>" for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(dict)
def html_dict(d):
    items = (f"<li>{k}={v}</li>" for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [17]:
htmlize.registry

{object: <function __main__.htmlize(a)>,
 int: <function __main__.html_int(a)>,
 decimal.Decimal: <function __main__.html_real(r)>,
 float: <function __main__.html_real(r)>,
 str: <function __main__.html_str(s)>,
 tuple: <function __main__.html_sequence(l)>,
 set: <function __main__.html_sequence(l)>,
 list: <function __main__.html_sequence(l)>,
 dict: <function __main__.html_dict(d)>}

In [10]:
htmlize(5)

'5(<i>0x5</i>)'

In [11]:
htmlize([1, 2, 3])

'<ul>\n<li>1</li>\n<li>2</li>\n<li>3</li>\n</ul>'

In [31]:
class SingleDispatch:
    def __init__(self, fn):
        self._registry = {}
        self._registry[object] = fn
    
    def __call__(self, arg):
        return self._registry.get(type(arg), self._registry[object])(arg)
    
    def register(self, type_):
        def inner(fn):
            self._registry[type_] = fn
            return fn
        return inner

In [32]:
@SingleDispatch
def htmlize(a):
    return escape(str(a))


In [33]:
@htmlize.register(int)
def html_int(a):
    return f"{a}(<i>{str(hex(a))}</i>)"

@htmlize.register(float)
@htmlize.register(Decimal)
def html_real(r):
    return f"{a:.2f}"

@htmlize.register(str)
def html_str(s):
    return s.replace('\n', '<br/>\n')

@htmlize.register(list)
@htmlize.register(set)
@htmlize.register(tuple)
def html_sequence(l):
    items = (f"<li>{html_escape(item)}</li>" for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(dict)
def html_dict(d):
    items = (f"<li>{k}={v}</li>" for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [ ]:
from functools import singledispatch

### Exercises

✅ Easy                
Write a decorator `@logger` that prints "Calling function..." before calling the function and "Function called." after.

🔄 Medium       
Write a decorator `@timer` that times how long a function takes to execute and prints it.

Stack `@logger` and `@timer` on a function and observe the order of output.


In [54]:
def logger(fn):
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        print(f"Starting {fn.__name__}")
        res = fn(*args, **kwargs)
        print(f"{fn.__name__} ended.")
        return res
    
    return inner

In [55]:
def timer(fn):
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        res = fn(*args, **kwargs)
        print(f"{fn.__name__} took {perf_counter() - start}s")
        return res
    
    return inner

In [72]:
from time import sleep

@timer
@logger
def sleeper(t: int):
    sleep(t)

In [59]:
sleeper(1)

Starting sleeper
sleeper ended.
sleeper took 1.0019509000121616s


Write `@authorize(role="admin")` that allows function execution only if `user.role == "admin"` (assume a user object is globally accessible).

In [93]:
user = 'admin'

In [89]:
def authorize(role='user'):

    def dec(fn):
        from functools import wraps

        @wraps(fn)
        def inner(*args, **kwargs):
            global user
            if user == role:
                return fn(*args, **kwargs)
            else:
                raise PermissionError('admin role required for that function')
        
        return inner
    return dec

In [90]:
@authorize(role='admin')
def sleeper(t):
    sleep(t)

In [94]:
sleeper(1)

Write a decorator `@singleton` that makes sure only one instance of the class is ever created.

In [96]:
def singleton(obj):
    instance = None

    def get_instance(*args, **kwargs):
        nonlocal instance
        if not instance:
            instance = obj(*args, **kwargs)
        return instance
    
    return get_instance


Implement a decorator to handle exceptions with a default value

In [14]:
def default_value_on_exception(val):

    def dec(fn):
        from functools import wraps

        @wraps(fn)
        def inner(*args, **kwargs):
            try:
                return fn(*args, **kwargs)
            except Exception:
                return val
            
        return inner
    return dec

In [15]:
@default_value_on_exception(15)
def errorer(a, b, c):
    print(a, b, c)
    raise Exception

In [16]:
errorer(1, 2, 3)

1 2 3


15